In [1]:
%matplotlib qt
from importlib import reload
import xanes_analysis as xa
xa = reload(xa)
import xanes_math as xm
xm = reload(xm)
import xanes_spectra_filters as xsf
xsf = reload(xsf)
import numpy as np
import h5py, os
import multiprocess as mp
import matplotlib.pyplot as plt

[numexpr.utils] INFO : Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[numexpr.utils] INFO : NumExpr defaulting to 8 threads.
[dxchange.reader] WARNING : astropy module not found


In [2]:
fn = '/media/xiao_usb/3D_trial_reg_scan_id_29773-29873_2020-06-01-20-38-09.h5'
f = h5py.File(fn, 'r')
data = f['/registration_results/reg_results/registered_xanes3D'][:, 0, 50:200, 420:550]
#data = f['/registration_results/reg_results/registered_xanes3D'][:, 0, :, :]*1e4
eng = f['/registration_results/reg_results/eng_list'][:]*1e3
f.close()

# ***xanes_spectra_filters***

In [9]:
plt.plot(eng, data[:, 50:70, 70:90].sum(axis=(1,2)))

In [3]:
xatest = xa.xanes_analysis(data, eng, 8348,
                           pre_es=None, pre_ee=-50,
                           post_es=100, post_ee=None,
                           edge_jump_threshold=3,
                           pre_edge_threshold=3)

In [41]:
xatest.fit_pre_edge()
xatest.fit_post_edge()
xatest.cal_pre_edge_sd()
xatest.cal_post_edge_sd()
xatest.cal_pre_edge_mean()
xatest.cal_post_edge_mean()

In [15]:
print(xatest.post_edge_fit_coef.shape)
print(xatest.spectrum.shape)

(2, 150, 130)
(101, 150, 130)


In [24]:
preedge_fit = xsf.cal_pre_edge_fit(eng, xatest.pre_edge_fit_coef)
postedge_fit = xsf.cal_post_edge_fit(eng, xatest.post_edge_fit_coef)

In [27]:
jump = xsf.cal_post_edge_fit(np.array([xatest.preset_edge_eng]), xatest.post_edge_fit_coef) - \
       xsf.cal_pre_edge_fit(np.array([xatest.preset_edge_eng]), xatest.pre_edge_fit_coef)

In [28]:
print(preedge_fit.shape)
print(eng.shape)
print(jump.shape)


(101, 150, 130)
(101,)
(150, 130)


In [34]:
plt.figure(1)
plt.imshow(jump)
plt.figure(2)
plt.imshow(xatest.pre_edge_sd_map)

In [19]:
print(xatest.post_edge_fit_coef[:, 0, 0])
print(preedge_fit[:, 0,0])
print((np.vstack((eng, np.ones(eng.shape[0]))).T).shape)

[-2.43115389e-07  1.96475630e-03]
[ 1.16017121 -1.14612614]
(101, 2)


In [14]:
t0 = (xatest.pre_edge_fit_coef * 
(np.vstack((eng, np.ones(eng.shape[0]))).T)
 [:, :, np.newaxis, np.newaxis]).sum(axis=1)

t1 = (xatest.post_edge_fit_coef * 
(np.vstack((eng, np.ones(eng.shape[0]))).T)
 [:, :, np.newaxis, np.newaxis]).sum(axis=1)

In [21]:
t.shape

(101, 150, 130)

In [18]:
plt.plot(eng, t0[:, 50:55, 70:75].mean(axis=(1,2)))
plt.plot(eng, t1[:, 50:55, 70:75].mean(axis=(1,2)))
plt.plot(eng, xatest.spectrum[:, 50:55, 70:75].mean(axis=(1,2)))

In [20]:
print(xatest.pre_es, xatest.pre_ee, xatest.post_es, xatest.preset_edge_eng)

8212.0029296875 8298 8448 8348


In [25]:
plt.plot(eng, preedge_fit[:, 50:55, 70:75].mean(axis=(1,2)))
plt.plot(eng, postedge_fit[:, 50:55, 70:75].mean(axis=(1,2)))
plt.plot(eng, xatest.spectrum[:, 50:55, 70:75].mean(axis=(1,2)))

In [42]:
t2 = xsf.edge_jump_filter(np.array([xatest.preset_edge_eng]), xatest.pre_edge_fit_coef, xatest.post_edge_fit_coef, 
                          xatest.pre_edge_sd_map, xatest.edge_jump_threshold)

In [45]:
plt.imshow(t2*data[50, :])

In [72]:
t3 = xsf.fitted_edges_filter(eng, xatest.pre_edge_fit_coef, xatest.post_edge_fit_coef,
                             xatest.pre_edge_sd_map, xatest.pre_edge_threshold)

In [73]:
print(t3.shape)
plt.imshow(t3*data[50, :])

(150, 130)


In [65]:
mask = np.any((xsf.cal_post_edge_fit(eng, xatest.post_edge_fit_coef) -
               xsf.cal_pre_edge_fit(eng, xatest.pre_edge_fit_coef)) 
                  > 6*xatest.pre_edge_sd_map, axis=0).astype(np.int8)

In [71]:
print(mask.shape)

(150, 130)


In [67]:
plt.imshow(data[50, :]*mask*t2)

# ***xanes.math***

In [3]:
ids = list(np.arange(40, 54))
rlt = xm.fit_peak_scipy(eng[ids], data[ids, :].reshape(len(ids), -1), 'lorentzian', (0.01, eng[46], 2), 
                        bnds=((-np.inf, eng[40], 0.01), (np.inf, eng[60], 100)), 
                        ftol = 1e-7, xtol = 1e-7, gtol = 1e-7,
                        jac = '3-point', method = 'trf')

Wed Jun 17 08:58:05 2020
Wed Jun 17 08:58:15 2020


In [4]:
print(len(rlt))
print(rlt[0].x)

19500
[-4.84895432e-04  8.34665279e+03  2.29731357e-01]


In [18]:
plt.figure(5)
dim = data.shape
xd = np.linspace(eng[ids][0], eng[ids][-1], eng[ids].shape[0]*100)
for ii in range(55, 65):
    for jj in range(70, 80):
        idx = ii*dim[2]+jj
        plt.plot(xd, xm.functions['lorentzian'](xd, rlt[idx].x[0], rlt[idx].x[1], rlt[idx].x[2]))
        #plt.plot(eng[ids], data[ids, :][:, ii, jj])

In [29]:
plt.figure(6)
#plt.plot(xd, xm.functions['lorentzian'](xd, rlt2[55:65, 70:80].x[0], rlt2[55:65, 70:80].x[1], rlt[idx].x[2]))
plt.plot(eng[ids], data[ids, :][:, 55:65, 70:80].sum(axis=(1,2)))

In [ ]:
rlt1 = np.array(rlt)
print(rlt1.shape)
rlt2 = np.array(rlt).reshape([dim[1], dim[2]])

In [14]:
func = xm.functions['lorentzian']
def find_peak(func, x, fvars):
    return np.argmax(func(x, fvars[0], fvars[1], fvars[2]))

xd = np.linspace(eng[ids][0], eng[ids][-1], eng[ids].shape[0]*100)
n_cpu = os.cpu_count()
with mp.Pool(n_cpu-1) as pool:
    peaks = pool.starmap(find_peak, [(func, xd, rlt[ii].x) for ii in np.int32(np.arange(len(rlt)))])
pool.close()
pool.join()

In [38]:
tem=list(xm.functions['lorentzian'](xd, rlt[0].x[0], rlt[0].x[1], rlt[0].x[2]))

In [9]:
print(len(peaks))
dim = data.shape
plt.imshow(np.array(peaks).reshape([dim[1], dim[2]]))

19500


In [12]:
xd = np.linspace(eng[ids][0], eng[ids][-1], eng[ids].shape[0]*100)
peaks = xm.find_peak_map('lorentzian', xd, rlt)
print(xd.shape)

Wed Jun 17 09:19:54 2020
Wed Jun 17 09:19:55 2020
(1400,)


In [13]:
print(len(peaks))
dim = data.shape
plt.imshow(np.array(peaks).reshape([dim[1], dim[2]]))

19500


In [15]:
co_xrf_per_ugcm2_per_ic = 0.00242 / 19
ic = 50000
a = 2e-5*(1e7/50) *ic 
print(a)

200000.0


In [3]:
with h5py.File('/media/Disk2/data/AIJUN_LI_Proposal_304474_June28_2019/2D_trial_reg_xanes_scan2_id_24164_2020-06-19-20-47-47.h5', 'r+') as f:
    imgs = f['/registration_results/reg_results/registered_xanes2D'][:, 400:700, 400:700]
    xanes2D_analysis_eng_list = f['/processed_XANES2D/proc_parameters/eng_list'][:]
    xanes2D_analysis_edge_eng = f['/processed_XANES2D/proc_parameters/edge_eng'][()]
    xanes2D_analysis_pre_edge_e = f['/processed_XANES2D/proc_parameters/pre_edge_e'][()]
    xanes2D_analysis_post_edge_s = f['/processed_XANES2D/proc_parameters/post_edge_s'][()]
    xanes2D_analysis_edge_jump_thres = f['/processed_XANES2D/proc_parameters/edge_jump_threshold'][()]
    xanes2D_analysis_edge_offset_thres = f['/processed_XANES2D/proc_parameters/edge_offset_threshold'][()]
    xanes2D_analysis_use_mask = f['/processed_XANES2D/proc_parameters/use_mask'][()]
    xanes2D_analysis_type = f['/processed_XANES2D/proc_parameters/analysis_type'][()]
    xanes2D_analysis_data_shape = f['/processed_XANES2D/proc_parameters/data_shape'][:]
    xanes2D_analysis_edge_0p5_fit_s = f['/processed_XANES2D/proc_parameters/edge_0p5_fit_s'][()]
    xanes2D_analysis_edge_0p5_fit_e = f['/processed_XANES2D/proc_parameters/edge_0p5_fit_e'][()]
    xanes2D_analysis_wl_fit_eng_s = f['/processed_XANES2D/proc_parameters/wl_fit_eng_s'][()]
    xanes2D_analysis_wl_fit_eng_e = f['/processed_XANES2D/proc_parameters/wl_fit_eng_e'][()]
    xanes2D_analysis_use_flt_spec = f['/processed_XANES2D/proc_parameters/flt_spec'][()]
    xanes2D_analysis_norm_fitting_order = f['/processed_XANES2D/proc_parameters/normalized_fitting_order'][()]
    xana = xa.xanes_analysis(imgs, xanes2D_analysis_eng_list, xanes2D_analysis_edge_eng, pre_ee=xanes2D_analysis_pre_edge_e, post_es=xanes2D_analysis_post_edge_s, edge_jump_threshold=xanes2D_analysis_edge_jump_thres, pre_edge_threshold=xanes2D_analysis_edge_offset_thres)


In [4]:
rlt = xana.fit_whiteline(xanes2D_analysis_wl_fit_eng_s, xanes2D_analysis_wl_fit_eng_e, model='lorentzian', 
                   fvars=(0.5, xanes2D_analysis_wl_fit_eng_s, 1), bnds=None, ftol=1e-7, xtol=1e-7, gtol=1e-7, 
                   jac='3-point', method='trf', ufac=100)

Sat Jun 20 19:56:22 2020
Sat Jun 20 19:57:19 2020
Sat Jun 20 19:57:19 2020
Sat Jun 20 19:57:24 2020


In [7]:
#rlt1 = np.array(rlt).reshape(imgs.shape[1:])
print(xana.peak_map.shape)

(300, 300)


In [11]:
eng = np.linspace(xanes2D_analysis_eng_list[45], xanes2D_analysis_eng_list[53], 800)
peak_mp = xm.find_peak_map('lorentzian', eng, rlt)

Sat Jun 20 16:09:57 2020
Sat Jun 20 16:11:15 2020


In [18]:
p = np.array(peak_mp).reshape(imgs.shape[1:])

In [9]:
plt.imshow(xana.peak_map)

# ***xanes_analysis***

In [4]:
xatest.find_e0()

In [5]:
plt.imshow(xatest.auto_e0)